In [1]:
import yfinance as yf
import pandas as pd
from prophet import Prophet
import numpy as np

class MinuteDataPredictor:
    def __init__(self, symbol, file_path):
        self.symbol = symbol
        self.file_path = file_path
        self.minute_data = None
    
    def fetch_minute_data(self):
        try:
            # Fetch historical data for the last x amount of days with x amount of minute intervals
            data = yf.download(self.symbol, period='1d', interval='60m')

            # Select only the 'Close' prices and reset the index to include 'Timestamp'
            self.minute_data = data[['Close']].reset_index()

            # Rename columns to match 'Timestamp' and 'Price'
            self.minute_data.rename(columns={'Datetime': 'Timestamp', 'Close': 'Price'}, inplace=True)

            # Format the timestamp column
            self.minute_data['Timestamp'] = self.minute_data['Timestamp'].dt.strftime("%Y-%m-%d %H:%M:%S")

            # Add empty columns for 'Prediction' and 'Accuracy'
            self.minute_data['Prediction_Prophet'] = None
            self.minute_data['Accuracy_Prophet'] = None
        
        except Exception as e:
            print(f"Error fetching minute data: {e}")

    def prophet_predictions(self, data):
        data = data[['Timestamp', 'Price']].copy()
        data.rename(columns={'Timestamp': 'ds', 'Price': 'y'}, inplace=True)
        model = Prophet()
        model.fit(data)
        future = model.make_future_dataframe(periods=1, freq='min')
        forecast = model.predict(future)
        next_minute_prediction = forecast['yhat'].iloc[-1]
        print(next_minute_prediction)
        return next_minute_prediction
    
    def calculate_accuracy(self, next_minute_prediction_prophet, i):
            correct_predictions = 0
            for j in range(1, i):
                
                actual_price = self.minute_data.loc[j, 'Price']
                actual_change = self.minute_data.loc[j + 1, 'Price'] - actual_price
                predicted_change = next_minute_prediction_prophet - actual_price
               # print("the actual price is : ", actual_price, "actual change = ",actual_change)
               # print("the prediction price (forecasting +1min) is : ", next_minute_prediction_prophet, "predicted change = ",predicted_change)
                
                if (predicted_change > 0 and actual_change > 0 ):
                    correct_predictions += 1

                if (predicted_change < 0 and actual_change < 0 ):
                    correct_predictions += 1

            if correct_predictions == 0 :
                return 0
            else:
                accuracy = (correct_predictions / i) * 100
                return accuracy

    def save_minute_data_with_predictions(self):
        try:
            self.fetch_minute_data()

            for i in range(1, len(self.minute_data) - 1):
                previous_data = self.minute_data.iloc[:i + 1].copy()
                
                # Make predictions using Prophet model
                next_minute_prediction_prophet = self.prophet_predictions(previous_data)
                self.minute_data.at[i, 'Prediction_Prophet'] = next_minute_prediction_prophet

                # Calculate accuracy for Prophet model
                accuracy_percentage_prophet = self.calculate_accuracy(next_minute_prediction_prophet, i)
                self.minute_data.at[i, 'Accuracy_Prophet'] = accuracy_percentage_prophet
                if len(previous_data) > 2:
                    print(f"{self.minute_data.at[i, 'Timestamp']}: "
                        f"Prediction (Prophet) - {next_minute_prediction_prophet}, "
                        f"Accuracy (Prophet) - {accuracy_percentage_prophet}%, "
                       )
            # Save data to a CSV file
            self.minute_data.to_csv(self.file_path, index=False)
            print(f"Minute data with predictions and accuracy based on both models saved to {self.file_path}")

        except Exception as e:
            print(f"Error: {e}")
predictor = MinuteDataPredictor(symbol='AAPL', file_path='minute_data_aapl_with_prediction_oop.csv')

predictor.save_minute_data_with_predictions()

[*********************100%%**********************]  1 of 1 completed


10:49:08 - cmdstanpy - INFO - Chain [1] start processing
10:49:13 - cmdstanpy - INFO - Chain [1] done processing


185.6123207353592


10:49:13 - cmdstanpy - INFO - Chain [1] start processing
10:49:16 - cmdstanpy - INFO - Chain [1] done processing


185.47275375473274
2024-02-13 11:30:00: Prediction (Prophet) - 185.47275375473274, Accuracy (Prophet) - 50.0%, 


...

185.291995616037
2024-02-13 12:30:00: Prediction (Prophet) - 185.291995616037, Accuracy (Prophet) - 66.66666666666666%, 


10:49:26 - cmdstanpy - INFO - Chain [1] start processing
10:49:40 - cmdstanpy - INFO - Chain [1] done processing


184.26816902260728
2024-02-13 13:30:00: Prediction (Prophet) - 184.26816902260728, Accuracy (Prophet) - 75.0%, 


10:49:41 - cmdstanpy - INFO - Chain [1] start processing
10:49:41 - cmdstanpy - INFO - Chain [1] done processing


183.6426802953583
2024-02-13 14:30:00: Prediction (Prophet) - 183.6426802953583, Accuracy (Prophet) - 80.0%, 
Minute data with predictions and accuracy based on both models saved to minute_data_aapl_with_prediction_oop.csv
